In [1]:
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# defining the server and the database
server = '(localdb)\mssqllocaldb' 
database = 'AdventureWorks2012'  

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE ='+ database +';\
    Trusted_Connection=yes;'
) 

### Question 1 - AdventureWorks DB - Sales.SalesOrderDetail table
### 1. How many items has Adventure Works sold in its history? (use [Sales].[SalesOrderDetail])

In [2]:
# Using Count() function and SUM() function to get the total number of sold items for Adventure Works

sql_query = pd.read_sql_query("""SELECT SUM(Sales.OrderQty) AS [Items Ordered], COUNT(1) AS [Number of Sales]
                                 FROM [AdventureWorks2012].[Sales].[SalesOrderDetail] AS Sales""",conn)
sql_query


,Items Ordered,Number of Sales
0,274914,121317


### Question 2 - AdventureWorks DB - Production.Product table
### 2. What is the average list price for AdventureWorks products?

In [3]:
# Using AVG() function to get the list price for Adventure Works products

sql_query = pd.read_sql_query("""SELECT AVG(Product.ListPrice) AS [Average List Price]
                                 FROM [AdventureWorks2012].[Production].[Product] AS Product""",conn)
sql_query



,Average List Price
0,438.6662


### Question 3 - AdventureWorks DB - Production.Product table
### 3. What is the average list price for AdventureWorks products (this time only include each
### price once even if more than one product have the same price)?

In [4]:
# Using DISTINCT within AVG() function to get the list price for Adventure Works products including each 
# price only once in calculation

sql_query = pd.read_sql_query("""SELECT AVG(DISTINCT Product.ListPrice) AS [Average List Price]
                                 FROM [AdventureWorks2012].[Production].[Product] AS Product""",conn)
sql_query

,Average List Price
0,437.4042


### Question 4 - AdventureWorks DB - Production.Product table
### 4. Get the average list price and average unique list price (previous 2 questions) by color of
### the product

In [5]:
# Using DISTINCT within AVG() function to get the list price for Adventure Works products including each 
# price only once in calculation

sql_query = pd.read_sql_query("""SELECT Product.Color AS [Grouped By Color],
                                 AVG(Product.ListPrice) AS [Average List Price],
                                 AVG(DISTINCT Product.ListPrice) AS [Average Unique List Price]
                                 FROM [AdventureWorks2012].[Production].[Product] AS Product
                                 GROUP BY Color""",conn)
sql_query

,Grouped By Color,Average List Price,Average Unique List Price
0,None,16.8641,65.9275
1,Black,725.1210,527.5882
2,Blue,923.6792,825.2985
3,Grey,125.0000,125.0000
4,Multi,59.8650,49.6566
5,Red,1401.9500,1332.6425
6,Silver,850.3053,726.2907
7,Silver/Black,64.0185,61.1900
8,White,9.2450,9.2450
9,Yellow,959.0913,991.7562


### Question 5 - AdventureWorks DB - Sales.SalesOrderHeader table
### 5. How much tax was collected on valentine’s day in 2008 from internet sales? (use
### [Sales].[SalesOrderHeader]). How many sales transactions happened that day?

In [6]:
# Using the date 02/14/2008 for Valentines date and SELECT-ing TaxAmt column the total tax collected for the day
# Using Count(1) to calculate the number of transactions
# Using OnlineOrderFlag = 1 to filter results in WHERE clause

sql_query = pd.read_sql_query("""SELECT SUM(Sales.TaxAmt) AS [Total Tax Amount],
                                 MIN(Sales.TaxAmt) AS [Minimum Tax Amount],
                                 MAX(Sales.TaxAmt) AS [Maximum Tax Amount],
                                 COUNT(1) AS [Number of transactions]
                                 FROM [AdventureWorks2012].[Sales].[SalesOrderHeader] AS Sales
                                 WHERE Sales.OnlineOrderFlag = 1 AND Sales.OrderDate = '2008-02-14'""",conn)
sql_query

,Total Tax Amount,Minimum Tax Amount,Maximum Tax Amount,Number of transactions
0,4182.9,0.3192,193.5248,63


### Question 6 - AdventureWorks DB - Sales.SalesOrderHeader table
### 6. Get the “total due” per day for the Month of July 2008

In [7]:
# Using the date 2008-02-01 and 2008-02-29 for month of February to get the SUM() for TotalDue column

sql_query = pd.read_sql_query("""SELECT Sales.OrderDate, SUM(Sales.TotalDue) AS [Total Due Amount]
                                 FROM [AdventureWorks2012].[Sales].[SalesOrderHeader] AS Sales
                                 WHERE Sales.OrderDate BETWEEN '2008-02-01' AND '2008-02-29'
                                 GROUP BY Sales.OrderDate
                                 ORDER BY Sales.OrderDate""",conn)
sql_query

,OrderDate,Total Due Amount
0,2008-02-01,3.108772e+06
1,2008-02-02,5.459855e+04
2,2008-02-03,6.295564e+04
3,2008-02-04,5.404213e+04
4,2008-02-05,6.060804e+04
5,2008-02-06,5.246321e+04
6,2008-02-07,7.750763e+04
7,2008-02-08,7.339820e+04
8,2008-02-09,4.663015e+04
9,2008-02-10,5.349598e+04


### Question 7 - AdventureWorks DB - Sales.SalesOrderHeader table
### 7. Repeat previous question but disregard days where the “Total Dues” is less than 2000 dollars.

In [8]:
# Using the date 2008-02-01 and 2008-02-29 for month of February to get the SUM() for TotalDue column
# Exclude any records where Total Due is less than 2000 from query

sql_query = pd.read_sql_query("""SELECT Sales.OrderDate, SUM(Sales.TotalDue) AS [Total Due Amount]
                                 FROM [AdventureWorks2012].[Sales].[SalesOrderHeader] AS Sales
                                 WHERE Sales.OrderDate BETWEEN '2008-02-01' AND '2008-02-29' AND
                                 Sales.TotalDue >= 2000
                                 GROUP BY Sales.OrderDate
                                 ORDER BY Sales.OrderDate""",conn)
sql_query


,OrderDate,Total Due Amount
0,2008-02-01,3.044740e+06
1,2008-02-02,3.958243e+04
2,2008-02-03,4.465974e+04
3,2008-02-04,3.981177e+04
4,2008-02-05,3.403688e+04
5,2008-02-06,3.173981e+04
6,2008-02-07,4.765637e+04
7,2008-02-08,5.053208e+04
8,2008-02-09,2.935978e+04
9,2008-02-10,3.654561e+04


### Question 8 - AdventureWorks DB - Person.BusinessEntityAddress table
### 8. Based on [BusinessEntityAddress] table, list the BusinessEntityID which have multiple addresses

In [9]:
# Using SELECT to query all BusinessEntityAddress records with multiple addresses
# Using Having clause to filter only for records with more than 1 address

sql_query = pd.read_sql_query("""SELECT Bea.BusinessEntityID, COUNT(Bea.AddressTypeID)
                                 FROM [AdventureWorks2012].[Person].[BusinessEntityAddress] AS Bea
                                 GROUP BY Bea.BusinessEntityID
                                 HAVING COUNT(Bea.AddressTypeID) > 1""",conn)
sql_query


,BusinessEntityID,
0,332,2
1,430,2
2,472,2
3,616,2
4,932,2
5,1028,2
6,1126,2
7,1170,2
8,1204,2
9,1330,2


### Question 9 - AdventureWorks DB - Person.BusinessEntityAddress table
### 9. Returns the total sales (order header) for each sales person

In [10]:
# Using SELECT to query the totals sales for each sales person using TotalDue Column
# Using GROUP BY to group SalesPersonID to ther sales amount and YEAR(Sales.OrderDate) to get the yearly sales
# Using ORDER BY Sales.SalesPersonID then by YEAR(Sales.OrderDate) to group sales by sales person for each year

sql_query = pd.read_sql_query("""SELECT Sales.SalesPersonID, SUM(Sales.TotalDue) AS [Sales Amount],
                                 YEAR(Sales.OrderDate) AS [Sales Year]
                                 FROM [AdventureWorks2012].[Sales].[SalesOrderHeader] AS Sales
                                 GROUP BY Sales.SalesPersonID, YEAR(Sales.OrderDate)
                                 ORDER BY Sales.SalesPersonID, YEAR(Sales.OrderDate)""",conn)
sql_query


,SalesPersonID,Sales Amount,Sales Year
0,NaN,3.609343e+06,2005
1,NaN,7.216030e+06,2006
2,NaN,1.081912e+07,2007
3,NaN,1.079684e+07,2008
4,274.0,3.256792e+04,2005
...,...,...,...
57,289.0,4.705252e+06,2007
58,289.0,2.037153e+06,2008
59,290.0,9.563256e+05,2006
60,290.0,2.582651e+06,2007
